## LCEL 인터페이스


사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain 을 생성합니다.

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

## stream: 실시간 출력


이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [4]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 여러 가지 다른 형태의 커뮤니케이션 방식을 통합하여 정보를 전달하는 방법이다. 이는 텍스트, 음성, 이미지, 동영상 등 다양한 매체를 활용하여 사용자에게 풍부한 경험을 제공한다. 멀티모달은 사용자들에게 대화형 인터페이스를 통해 편리하고 효과적인 소통 경로를 제공한다.

## invoke: 호출


`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [5]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 인공지능 챗봇으로 자연어 이해와 생성 기술을 사용하여 대화 상대와 상호작용을 합니다. 사용자의 질문에 대답하거나 대화를 이어나가는 등 다양한 작업을 수행할 수 있습니다. ChatGPT는 학습 데이터를 기반으로 문맥을 이해하고 심층적인 대화를 제공합니다.'

## batch: 배치(단위 실행)


함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.

In [6]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 인공 지능 기술을 이용하여 대화 상대와 대화할 수 있는 자연어 처리 모델이다. 사용자의 질문이나 입력에 대해 자연스러운 대화를 제공하며 다양한 주제에 대해 학습하고 지식을 공유할 수 있다. ChatGPT는 실시간 채팅, 정보 제공, 상담 및 엔터테인먼트 등 다양한 용도로 활용될 수 있다.',
 'Instagram은 소셜 미디어 플랫폼으로, 사용자들이 사진과 동영상을 공유하고 다른 사람들과 소통하는 공간입니다. 다양한 필터와 편집 기능을 통해 사용자들은 자신의 콘텐츠를 보다 멋지게 표현할 수 있습니다. 또한 해시태그를 활용하여 관심 주제나 트렌드에 맞는 콘텐츠를 찾을 수 있습니다.']

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

`config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [7]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 인공지능 챗봇으로 대화를 통해 사용자에게 도움을 제공합니다. 이 모델은 자연어 처리 기술을 기반으로 작동하며, 다양한 주제에 대해 대화를 이어나갈 수 있습니다. 사용자들은 ChatGPT를 통해 질문에 답변을 얻거나 대화를 나눌 수 있습니다.',
 '인스타그램은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 다양한 필터와 효과로 꾸밀 수 있습니다. 또한 팔로워들과 소통하며 콘텐츠를 공유하고, 다른 사람들의 포스트를 좋아하고 댓글을 남길 수 있습니다. 인스타그램은 시각적인 콘텐츠를 중심으로 한 플랫폼으로, 사용자들은 자신의 취향과 관심사를 시각적으로 표현하며 소통할 수 있습니다.',
 '멀티모달은 여러 가지 다양한 형태의 수단을 결합하여 이용자에게 최적의 이용 경로를 제공하는 서비스이다. 대중교통, 자전거, 자동차 등을 융합하여 효율적인 이동을 도와준다. 스마트폰 앱을 통해 실시간 정보를 확인하고 예약할 수 있어 편리하고 효율적인 이동이 가능하다.',
 '프로그래밍은 컴퓨터에게 일련의 명령을 내리는 과정이다. 이를 통해 원하는 기능을 구현하고 문제를 해결할 수 있다. 프로그래밍 언어를 사용하여 알고리즘을 작성하고 실행시켜 원하는 결과를 얻을 수 있다.',
 '머신러닝은 컴퓨터가 데이터를 분석하고 패턴을 학습하여 예측하거나 결정을 내리는 인공지능 기술이다. 머신러닝은 지도학습, 비지도학습, 강화학습으로 분류되며 다양한 분야에서 활용되고 있다. 머신러닝을 통해 데이터를 신속하고 정확하게 분석하여 의사결정을 지원하는데 도움을 줄 수 있다.']

## async stream: 비동기 스트림


함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [8]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 전 세계에서 가장 인기 있는 동영상 공유 플랫폼이며, 사용자들은 다양한 주제의 동영상을 업로드하고 시청할 수 있다. 광고 수익을 얻을 수 있는 유튜버들이 많이 활동하고 있으며, 개인 및 기업들은 YouTube를 통해 브랜드 홍보 및 마케팅을 진행하고 있다. 사용자들은 구독, 좋아요, 댓글 등을 통해 동영상 제작자와 소통하며, 다양한 콘텐츠를 즐길 수 있다.

## async invoke: 비동기 호출


`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.


In [11]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

/var/folders/bt/50st22114q937kdvskmgksd80000gn/T/ipykernel_2608/209754833.py:2: RuntimeWarning: coroutine 'RunnableSequence.ainvoke' was never awaited
  my_process = chain.ainvoke({"topic": "NVDA"})


In [12]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

'NVDA는 미국의 컴퓨터 그래픽스 기술 기업인 NVIDIA Corporation의 주식을 나타내는 심볼이다. 이 회사는 주로 그래픽 처리 장치와 인공지능 기술을 개발하여 전 세계적으로 널리 알려져 있다. NVDA 주식은 기술 산업의 성장과 미래 기술에 대한 투자에 관심이 있는 투자자들 사이에서 인기가 높다.'

## async batch: 비동기 배치


함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있습니다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [13]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "Toss"}, {"topic": "Tmap"}, {"topic": "Tesla"}]
)

In [14]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['Toss는 던지다라는 동사로, 공을 공중으로 던지는 동작을 가리킨다. 스포츠나 게임에서 경기 시작 전에 이루어지는 동작으로, 공을 공중으로 던져 상대편에게 전달하거나 시작하는 시그널로 사용된다. 또한, 동전을 던져 앞뒷면을 결정하는 행위도 Toss라고 부른다.',
 'TMap은 삼성전자에서 개발한 지도 서비스 플랫폼으로, 다양한 교통수단의 실시간 도착 정보와 길 안내 등을 제공합니다. TMap은 사용자의 위치를 기반으로 교통 정보를 제공하고, 길찾기와 경로 안내 기능을 제공하여 편리한 이동을 도와줍니다. 또한 TMap은 편의 시설 정보와 주변 관광지 정보도 제공하여 다양한 정보를 한 곳에서 확인할 수 있습니다.',
 '1. Tesla는 전기 자동차 및 에너지 저장 솔루션을 제공하는 미국의 기업이다.\n2. 창업자 일론 머스크의 비전에 따라 혁신적인 기술과 디자인으로 소비자들에게 혁신적인 제품을 제공하고 있다.\n3. Tesla의 자동차는 전 세계적으로 인기를 끌며 친환경적이고 혁신적인 기술을 통해 자동차 산업을 변화시키고 있다.']

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행합니다.


`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.


In [15]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메서드를 호출합니다.

이때, `country`이라는 키에 `대한민국`라는 값을 가진 딕셔너리를 인자로 전달합니다.


In [16]:
# chain1 를 실행합니다.
chain1.invoke({"country": "몽골"})

'몽골의 수도는 울란바토르입니다.'

이번에는 `chain2.invoke()` 를 호출합니다. `country` 키에 다른 국가인 `미국` 을 전달합니다.


In [17]:
# chain2 를 실행합니다.
chain2.invoke({"country": "자메이카"})

'자메이카의 면적은 10,991 제곱 킬로미터 입니다.'

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행합니다.

이 예제에서는 `대한민국`라는 주제를 `invoke` 메서드에 전달하여 실행합니다.


In [18]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "북한"})

{'capital': '북한의 수도는 평양입니다.', 'area': '북한의 면적은 약 120,540km² 입니다.'}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.


`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.


In [19]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "남아공"}, {"country": "칠레"}])

['남아공의 수도는 프리토리아(Pretoria)입니다.', '칠레의 수도는 사티아고입니다.']

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(batch)를 수행합니다.

이 예시에서는 `대한민국`와 `미국`라는 두 가지 국가에 대한 처리를 요청합니다.


In [20]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 면적은 약 100,363 제곱 킬로미터입니다.', '미국의 면적은 약 9,826,675 km² 입니다.']

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`와 `미국` 두 나라에 대한 데이터를 배치 처리합니다.


In [21]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "일본"}, {"country": "중국"}])

[{'capital': '일본의 수도는 도쿄입니다.', 'area': '일본의 총 면적은 약 377,975 km² 입니다.'},
 {'capital': '중국의 수도는 베이징(北京)입니다.', 'area': '중국의 면적은 약 9,596,961 제곱 킬로미터입니다.'}]